In [1]:
import os
import pandas as pd
import numpy as np
import pydicom as dcm
import SimpleITK as sitk
import ProstateNetLoaders
pth = "/home/nst/Desktop/Procancer/Datasets/PROSTATENET-LESIONS/Datasetv0.3/Patients/PCa-239402179041957440623792026735212837616"
metadata= pd.read_csv("/home/nst/Desktop/Dimitris_Deep_Learning/Lesion_Radiomics/results.csv", 
                 names=["patient_id", "study_uid", "series_uid", "series_type", "series_type_heuristics"])

# 1. ALL in one Loader

In [18]:
a = ProstateNetLoaders.ExecuteLoader.Execute(pth, metadata,  Heuristics = True) #  Patient's Path, Metadata Path, choose between Heuristics (True) or AI (False)
a.LoadArrays(orientation="AX", seq="T2")
pat,ann = a.GetItems() # returns 2 dictionaries the patient and annotation dictionaries.
#They contain patient series and corresponding annotation respectively. Kyes are series, values are image arrays
print(pat.keys(), ann.keys()) 

dict_keys(['1.3.6.1.4.1.58108.1.87175058762966248438729854597427208030']) dict_keys(['1.3.6.1.4.1.58108.1.87175058762966248438729854597427208030'])


# 2. Components

# (a) series Loader

In [ ]:
ImObj = ProstateNetLoaders.SeriesPathLoaders.ProstateNetPathLoaders(pth)
ImObj.SeriesLoader() # loads the series dictionaries ({series name : series path})
ImObj.LoadObjects() # create sitk objects to extract information from
ImObj.LoadSeriesDescription() # returns the description of each series in a dict for a single patient
ser, descr = ImObj.GetSitkObjSerDescr() # get the paths to the corresponding series


In [ ]:
ser 

In [ ]:
descr

# (b) selection of the sequence

In [ ]:
ses = ProstateNetLoaders.SeriesPathLoaders.SequenceSelectorAI(pth, metadata)
ses.SetSeriesSequences(orientation="AX")
ser_dicts = ses.GetSeriesSequences()
ser_dicts


# (c) array and sitk image object loading

In [ ]:
series = {"1.3.6.1.4.1.58108.1.245617002898264363567933084613617434228":ser["1.3.6.1.4.1.58108.1.245617002898264363567933084613617434228"]}

arrobj = ProstateNetLoaders.SeriesPathLoaders.ArrayLoad(series)
arrobj.LoadITKobjects()
arr =arrobj.GetArray()
im = arrobj.GetImobj()
arr["1.3.6.1.4.1.58108.1.245617002898264363567933084613617434228"].shape


# (d) Segmentations Loading

In [ ]:
ld = ProstateNetLoaders.SegmentationLoaders.SegmentationLoader(pth, metadata)
ms  = ld.LoadMaskPath()
ld.SetOrderFiles()
ld.SetPosMask()
Anno = ld.MatchAnno()
Anno.shape

In [ ]:
import plotly.express as px

fig = px.imshow(arr['1.3.6.1.4.1.58108.1.245617002898264363567933084613617434228'] + 800*Anno, animation_frame=0,binary_string=True)
fig.show()

-------------

In [ ]:
pth_batch = "/home/nst/Desktop/Procancer/Datasets/PROSTATENET-LESIONS/Datasetv0.3/Patients"
patients = {}
Sequence = "T2"
T2_absence = []
for patient in os.listdir(pth_batch):
    pat = os.path.join(pth_batch,patient)
    a = ProstateNetLoaders.ExecuteLoader.Execute(pat, metadata)
    try:
        a.LoadArrays(orientation="AX", seq=Sequence)
        pat,ann = a.GetItems()
        patients.update({patient:{Sequence:np.array(list(pat.values())[0]),"Lesion": np.array(list(ann.values())[0])}})
    except: 
        T2_absence.append(patient)
        continue

In [ ]:
len(patients)

In [ ]:
T2_absence